In [1]:
import os
from optparse import OptionParser
import sys
import numpy as np
import pandas as pd
import json
import yaml

In [2]:
sys.path.insert(0, '/home/jgutman/mvesc/ETL')
sys.path.insert(0, '/home/jgutman/mvesc/Features')
sys.path.insert(0, '/home/jgutman/mvesc/Models_Results')

In [3]:
!git branch

* add_cross_val3_and_bootstrapping
  adding_allTeachers_to_database
  copy_df2postgres_file_to_utility
  error_analysis
  generate_normalized_test_scores
  gpa_generation
  intervention_dataETL_feature_generation
  master
  mobility_features
  week_update_model_testing


In [4]:
from mvesc_utility_functions import *

In [5]:
from sklearn.cross_validation import *

In [6]:
from estimate_prediction_model import *

In [7]:
model_options = read_in_yaml('/home/jgutman/mvesc/Models_Results/model_options_expand_features.yaml')

In [23]:
model_options['features_included'] = {
    'demographics': ['ethnicity', 'gender'], 
    'absence': ['absence*', 'absence_unexcused*', 'tardy*', 'tardy_unexcused*', 'medical*', 
                'absence_consec*', 'tardy_consec*'],
    'grades': ['gpa*', 'language_gpa*', 'stem_gpa*', 'humanities_gpa*', 'art_gpa*', 'health_gpa*', 
        'future_prep_gpa*', 'interventions_gpa*', 'language_gpa*', 'num_language_classes*', 'num_stem_classes*',
        'num_humanities_classes*', 'num_art_classes*', 'num_health_classes*', 'num_future_prep_classes*',
        'num_interventions_classes*', 'num_language_classes*', 'percent_passed_pf_classes*', 'num_pf_classes*',
        'gpa_district*'], 
    'mobility': ['n_addresses_to*', 'n_districts_to*', 'n_cities_to*', 'n_records_to*', 'avg_address_change_to*',
        'avg_district_change_to*', 'avg_city_change_to*', 'street_transition_in*', 'district_transition_in*',
        'city_transition_in*', 'mid_year_withdraw*'],
    'snapshots': ['disadvantagement*', 'disability*', 'gifted*', 'iss*', 'oss*', 'limited_english*', 
        'special_ed*', 'discipline_incidents*']
}

In [24]:
outcome_plus_features = build_outcomes_plus_features(model_options)

In [25]:
outcome_plus_features.dropna(subset=[model_options['outcome_name'],
        model_options['cohort_grade_level_begin']], inplace=True)

In [26]:
train, test = temporal_cohort_test_split(outcome_plus_features,
            model_options['cohort_grade_level_begin'],
            model_options['cohorts_held_out'],
            model_options['cohorts_training'])

In [27]:
print(train.shape)
print(test.shape)

(2127, 267)
(2009, 267)


In [28]:
train_X = train.drop([model_options['outcome_name'],
        model_options['cohort_grade_level_begin']],axis=1)
test_X = test.drop([model_options['outcome_name'],
        model_options['cohort_grade_level_begin']],axis=1)
train_y = train[model_options['outcome_name']]
test_y = test[model_options['outcome_name']]

In [29]:
# do missing value feature imputation here
train_X, test_X = impute_missing_values(train_X, test_X,
    model_options['missing_impute_strategy'])
assert (all(train_X.columns == test_X.columns)),\
    "train and test have different columns"